# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, **влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок**. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования**

Проверить гипотезу - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок

Ответить на вопросы (проверить гипотезы):

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Обзор данных

Импортирую библиотеку pandas, читаю датасет и вывожу первые 5 строк таблицы df

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Получаю общую информацию о таблице

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов с разными типами данных.

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок нет нарушений стиля.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о клиентах банка. Данные рассказывают о клиенте: его семейное положение, количество детей, общий трудовой стаж, возраст, образование, пол, ежемесячный жоход и т.д.

Предварительно можно утверждать, что, данных достаточно для проверки гипотезы. Но встречаются пропуски в `total_income` и `days_employed`. В столбце `education` разный регистр в данных. А также аномальные отрицательные значения в `days_employed`.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Сейчас же возьмем  столбец ежемесячного дохода `total_income` и заполним пропущенные значения в нём медианным значением по столбцу:
* опишем, какие пропущенные значения обнаружили;
* проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
* приведем возможные причины появления пропусков в данных;
* объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [4]:
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


В столбце `total_income` вместо показателей ежемесячного дохода (количественных данных) встречаются значения NaN. Поcчитаем, сколько в таблице пропущенных значений

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками

In [6]:
df.isna().sum() / df.shape[0]

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

доля пропущенных значений составляет **~10%** от общего количества записей.

**возможные причины появления пропусков в данных:**

* ошибка при выгрузке базы данных
* доход не указан клиентом

In [7]:
pd.options.display.float_format = '{:,.1f}'.format

display(df['total_income'].min()) # минимальный ежемес доход
display(df['total_income'].max()) # максимальный ежемес доход
display(df['total_income'].sort_values(ascending=False).head(30)) # сортировка по убыванию дохода
display(df['total_income'].mean()) # среднее значение
display(df['total_income'].median()) # медианное значение


20667.26379327158

2265604.028722744

12412   2,265,604.0
19606   2,200,852.2
9169    1,726,276.0
20809   1,715,018.4
17178   1,711,309.3
17503   1,597,613.5
18368   1,551,152.9
18353   1,427,934.5
15268   1,350,245.6
11071   1,286,281.0
2224    1,278,622.6
7447    1,240,165.0
19813   1,223,042.5
15211   1,172,459.8
13090   1,128,836.3
9871    1,103,455.4
10004   1,097,954.7
18766   1,092,608.2
19338   1,091,627.6
1590    1,089,120.3
15047   1,058,193.5
4415    1,030,899.2
18347   1,027,742.3
9156    1,006,468.1
4129    1,004,476.4
10613     997,014.3
8299      967,425.4
13926     958,434.6
5289      920,897.7
3097      910,451.5
Name: total_income, dtype: float64

167422.30220817294

145017.93753253992

**Заполнить пропуски медианным значением** — лучшее решение для количественных переменных в данном случае, так как cреднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Конкретно в данном случае разница между средним арифметическим и медианным значением не столь существенна. Однако расчет ежемесячного дохода медианным значением принято считать наиболее объективным и справедливым. Наилучшим решением будет расчет медианного значения в привязке к типу занятости.

заполним пропущенные значения в нём медианным значением по столбцу `total_income`:

In [8]:
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [9]:
#df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Заполняем пропуски с группировкой по переменной тип занятости: 

In [10]:
for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'total_income'].median()
    print(income_type, median)
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type), 'total_income'] = median

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995


Убедимся, что в стобце `total_income` не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [11]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в days_employed медианными значениями по этому столбцу.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [12]:
df['days_employed'].sample(15)

10958   352,724.2
11670    -3,489.5
12160   395,491.4
12382    -5,424.5
2726     -1,871.0
20078    -2,968.5
10273      -965.8
3377     -5,060.8
710      -3,040.4
7348     -3,180.7
13322    -1,056.3
18517      -537.3
16162    -5,178.4
19437      -216.8
3197     -1,598.6
Name: days_employed, dtype: float64

Исправляем отрицательные значения в положительные:

In [13]:
df['days_employed'] = df['days_employed'].abs()

Проверим аномалии в других столбцах:

In [14]:
for row in df: 
  display(df[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

142.3        1
1,849.6      1
886.3        1
2,539.5      1
390,575.0    1
            ..
1,394.3      1
2,325.7      1
4,086.4      1
1,259.5      1
1,636.4      1
Name: days_employed, Length: 19351, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

...

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

142,594.4    1105
172,358.0     509
118,514.5     414
150,447.9     147
499,163.1       2
             ... 
101,516.6       1
239,154.2       1
165,009.7       1
94,270.0        1
189,255.3       1
Name: total_income, Length: 19353, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Исправляем значения -1 на 1 и 20 на 2 в столбце `children`

In [15]:
df['children'] = df['children'].replace(-1,1)
df['children'] = df['children'].replace(20,2)

заменим значения 0 в столбце `gender` на медианые

In [16]:
df['dob_years'] = df['dob_years'].replace(0 , df['dob_years'].median())

(?не совсем понимаю что значит XNA это аномалия или норма?!) Заменим значение XNA  в стобце `gender` на пустое:

In [17]:
df.loc[df['gender'] == 'XNA', 'gender'] = ''

заполняем пропуски в `days_employed` медианными значениями по этому столбцу.

In [18]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

Убеждаемся, что больше нет пропусков и аномалий:

In [19]:
display(df.isna().sum()) # проверка пропусков

for row in df: 
  display(df[row].value_counts()) # проверка значений по столбцам

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

2,194.2      2175
329,951.6       1
886.3           1
2,539.5         1
390,575.0       1
             ... 
1,320.8         1
1,394.3         1
2,325.7         1
4,086.4         1
1,636.4         1
Name: days_employed, Length: 19351, dtype: int64

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

...

F    14236
M     7288
         1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

142,594.4    1105
172,358.0     509
118,514.5     414
150,447.9     147
499,163.1       2
             ... 
101,516.6       1
239,154.2       1
165,009.7       1
94,270.0        1
189,255.3       1
Name: total_income, Length: 19353, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Аномалии связанные с дубликатами, регистром и категоризацией исправим далее в следущих шагах

In [20]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,"21,525.0","21,525.0","21,525.0","21,525.0","21,525.0","21,525.0","21,525.0"
mean,0.5,"60,378.0",43.5,0.8,1.0,0.1,"165,225.3"
std,0.8,"133,257.6",12.2,0.5,1.4,0.3,"98,043.7"
min,0.0,24.1,19.0,0.0,0.0,0.0,"20,667.3"
25%,0.0,"1,025.6",34.0,1.0,0.0,0.0,"107,798.2"
50%,0.0,"2,194.2",42.0,1.0,0.0,0.0,"142,594.4"
75%,1.0,"4,779.6",53.0,1.0,1.0,0.0,"195,549.9"
max,5.0,"401,755.4",75.0,4.0,4.0,1.0,"2,265,604.0"


### Шаг 2.3. Изменение типов данных.

Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype().

In [21]:
df['total_income'].dtypes

dtype('float64')

Заменим вещественный тип данных в столбце `total_income` на целочисленный, с помощью метода `astype()`.

In [22]:
df['total_income'] = df['total_income'].astype('int')

проверяем тип:

In [23]:
df['total_income'].dtypes

dtype('int64')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [25]:
df['days_employed'] = df['days_employed'].astype('int')

In [26]:
df['days_employed'].dtypes

dtype('int64')

### Шаг 2.4. Удаление дубликатов.

Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.
После удаления дубликатов сделайте следующее:
* поясните, как выбирали метод для поиска и удаления дубликатов в данных;
* приведите возможные причины появления дубликатов.

Проверяем полные дубликаты:

In [27]:
df.duplicated().sum()

55

Удаляем полные дубликаты:

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

Проверяем дубликаты по столбцам:

In [29]:
for row in df: 
  display(df[row].value_counts()) 

0    14106
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

2194      2125
133         16
327         16
438         15
223         14
          ... 
4507         1
376084       1
349465       1
10636        1
2049         1
Name: days_employed, Length: 9086, dtype: int64

42.0    696
35.0    616
40.0    607
41.0    606
34.0    601
38.0    597
33.0    581
39.0    572
31.0    559
36.0    554
44.0    545
29.0    544
30.0    538
48.0    537
37.0    536
50.0    513
43.0    512
32.0    509
49.0    508
28.0    503
45.0    497
27.0    493
56.0    484
52.0    484
47.0    477
54.0    476
46.0    473
53.0    459
57.0    456
58.0    456
51.0    448
55.0    443
59.0    443
26.0    408
60.0    374
25.0    357
61.0    354
62.0    349
63.0    269
24.0    264
64.0    262
23.0    253
65.0    194
22.0    183
66.0    182
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

среднее                13704
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

1    15187
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

...

F    14188
M     7281
         1
Name: gender, dtype: int64

сотрудник          11090
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

0    19729
1     1741
Name: debt, dtype: int64

142594    1076
172357     504
118514     395
150447     145
144533       3
          ... 
179138       1
61724        1
99269        1
151371       1
126820       1
Name: total_income, Length: 18608, dtype: int64

свадьба                                   793
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

переводим education в нижний регистр:

In [30]:
df['education'] = df['education'].str.lower()

In [31]:
df.duplicated().sum()

17

проверяем значения столбца:

In [32]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Также значения дублируются в столбце `purpose`. Но его мы разбираем далее

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создайте два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.

Удалите из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.

In [33]:
df_education = df.loc[:,['education','education_id']].drop_duplicates().reset_index(drop=True) # создаем новый датфрейм

In [34]:
df_education.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [35]:
df_family_status = df.loc[:,['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)  # создаем новый датфрейм

In [36]:
df_family_status.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [37]:
df = df.drop(['education', 'family_status'], axis = 1) # удаляем столбцы education и family_status

In [38]:
df.head() # проверяем новый вид исходного датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создайте столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

Создаем функцию, которая будет определять категории в зависимости от ежемесячного дохода:

In [39]:
def total_income_category(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'

Проверяем работу функции:

In [40]:
total_income_category(25000)

'E'

Создаем `total_income_category` столбец с категориями

In [41]:
df['total_income_category'] = df['total_income'].apply(total_income_category)

Проверяем наличие столбца:

In [42]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Статистика по категориям дохода:

In [43]:
df['total_income_category'].value_counts()

C    16031
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.
Вы можете использовать собственную функцию и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.

Изучаем данные в столбце `purpose`:

In [44]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Создаем функцию, которая будет определять категории взависимости от цели кредита:

In [45]:
def purpose_category(purpose):
    try:
        if 'жиль' in purpose or 'недвижим' in purpose:
            return 'операции с недвижимостью'
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'автомобил' in purpose:
            return 'операции с автомобилем'
        if 'образовани' in purpose:
            return 'получение образования'
    except:
        return 'без категории'

Проверяем работу функции:

In [46]:
purpose_category('свадьба')

'проведение свадьбы'

Создаем столбец категорий:

In [47]:
df['purpose_category'] = df['purpose'].apply(purpose_category)

Проверяем значеня категорий в столбце:

In [48]:
df['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2334
Name: purpose_category, dtype: int64

In [49]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


<b>Резюме</b>

В ходе предобработки данных были выполнены следующие шаги:

- Были заполнены пропуски общим медианным значением по столбцу total_income
- Проверены и исправлены аномалии в данных. 
- Изменены типы данных столбца total_income и days_employed. 
- Удалены дубликаты в столбце education, присвоен нижний регистр. 
- Создана два новых словаря education и family_status
- Проведена категоризация дохода и целе кредита. 

Большинство имеет доход от 50001 до 200000. Наиболее востребованная цель кредита - операции с недвижимостью.


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

Построим сводную таблицу для ответа на вопрос:

In [50]:
children_debt = df.pivot_table(index=['children'], values = 'debt', aggfunc='mean').sort_values(by = 'debt', ascending = False)
pd.options.display.float_format = '{:,.4f}'.format
children_debt

,debt
children,
4,0.0976
2,0.0949
1,0.0916
3,0.0818
0,0.0754
5,0.0000


**ВЫВОД:**
Наиболее рискованной категорией для выдачи кредитов по наличию и количеству детей являются семьи, в которых 4 ребенка. Доля нарушений графика платежей для них будет равна 9,76%. Лучшие заемщики для банка - те, у которых нет детей, доля проблемных кредитов 7,5%.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:

Строим сводную таблицу для ответа на вопрос со столбцом доли невозврата кредита:

In [51]:
family_debt = df.pivot_table(index=['family_status_id'], values = 'debt', aggfunc='mean').sort_values(by = 'debt', ascending = False)

display(family_debt)

display(df_family_status)

,debt
family_status_id,
4,0.0975
1,0.0932
0,0.0754
3,0.0711
2,0.0657


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


**ВЫВОД**

Неженатые и незамужние чаще других совершают просрочки по кредиту - 9.75%. Реже всех группа с семейным статусом вдовец / вдова - 6.5%


##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

Построим сводную таблицу для ответа на вопрос:

In [52]:
income_debt = df.pivot_table(index=['total_income_category'], values = 'debt', aggfunc='mean').sort_values(by = 'debt', ascending = False)

income_debt

,debt
total_income_category,
E,0.0909
C,0.0848
A,0.0800
B,0.0706
D,0.0600


**ВЫВОД**

Наименьший риск невозврата кредита в срок у категории D с доходом от 30001 до 50000. (6%)
Наиболее высокий риск невозврата кредита в срок у группы E с доходом до 30000 тысяч - 9.09%

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:

In [53]:
purpose_debt = df.pivot_table(index=['purpose_category'], values = 'debt', aggfunc='mean').sort_values(by = 'debt', ascending = False)

purpose_debt

,debt
purpose_category,
операции с автомобилем,0.0935
получение образования,0.0922
проведение свадьбы,0.0797
операции с недвижимостью,0.0723


##### **ВЫВОД**

Доля просрочек кредитов с целью "операции с недвижимостью самая низкая и составляет 7.23%. Самый высокий риск невозврата у кредитов с целью "операции с автомобилем" - 9.35%

In [55]:
# код ревьюера
df.pivot_table(index=['purpose_category'], values = 'debt', aggfunc=('count', 'sum', 'mean')).reset_index()

,purpose_category,count,mean,sum
0,операции с автомобилем,"4,308.0000",0.0935,403.0000
1,операции с недвижимостью,"10,814.0000",0.0723,782.0000
2,получение образования,"4,014.0000",0.0922,370.0000
3,проведение свадьбы,"2,334.0000",0.0797,186.0000


## Общий вывод:

**В ходе работы были выполнены следующие шаги:**

- Были заполнены пропуски общим медианным значением по столбцу total_income
- Проверены и исправлены аномалии в данных. 
- Изменены типы данных столбца total_income и days_employed. 
- Удалены дубликаты в столбце education, присвоен нижний регистр. 
- Создана два новых словаря education и family_status
- Проведена категоризация дохода и целе кредита. 

Большинство имеет доход от 50001 до 200000. Наиболее востребованная цель кредита - операции с недвижимостью.

**Проверены гипотезы:**

* Есть ли зависимость между наличием детей и возвратом кредита в срок?

Наиболее рискованной категорией для выдачи кредитов по наличию и количеству детей являются семьи, в которых 4 ребенка. Доля нарушений графика платежей для них будет равна 9,76%. Лучшие заемщики для банка - те, у которых нет детей, доля проблемных кредитов 7,5%.

* Есть ли зависимость между семейным положением и возвратом кредита в срок?


Неженатые и незамужние чаще других совершают просрочки по кредиту - 9.75%. Реже всех группа с семейным статусом вдовец / вдова - 6.5%

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Наименьший риск невозврата кредита в срок у категории D с доходом от 30001 до 50000. (6%)
Наиболее высокий риск невозврата кредита в срок у группы E с доходом до 30000 тысяч - 9.09%

* Как разные цели кредита влияют на его возврат в срок?

Доля просрочек кредитов с целью "операции с недвижимостью самая низкая и составляет 7.2%. Самый высокий риск невозврата у кредитов с целью "операции с автомобилем" - 9.3%

<b>Рекомендации: </b>

При построении модели кредитного скоринга рекомендуется учитывать все данные факторы, рассмотренные выше: наличие детей, семейное положение, уровень дохода, цель кредита.